In [1]:
import requests
from newspaper import Article
import time

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

article_urls = [
    "https://www.artificialintelligence-news.com/2023/05/23/meta-open-source-speech-ai-models-support-over-1100-languages/",
    "https://www.artificialintelligence-news.com/2023/05/18/beijing-launches-campaign-against-ai-generated-misinformation/"
    "https://www.artificialintelligence-news.com/2023/05/16/openai-ceo-ai-regulation-is-essential/",
    "https://www.artificialintelligence-news.com/2023/05/15/jay-migliaccio-ibm-watson-on-leveraging-ai-to-improve-productivity/",
    "https://www.artificialintelligence-news.com/2023/05/15/iurii-milovanov-softserve-how-ai-ml-is-helping-boost-innovation-and-personalisation/",
    "https://www.artificialintelligence-news.com/2023/05/11/ai-and-big-data-expo-north-america-begins-in-less-than-one-week/",
    "https://www.artificialintelligence-news.com/2023/05/11/eu-committees-green-light-ai-act/",
    "https://www.artificialintelligence-news.com/2023/05/09/wozniak-warns-ai-will-power-next-gen-scams/",
    "https://www.artificialintelligence-news.com/2023/05/09/infocepts-ceo-shashank-garg-on-the-da-market-shifts-and-impact-of-ai-on-data-analytics/",
    "https://www.artificialintelligence-news.com/2023/05/02/ai-godfather-warns-dangers-and-quits-google/",
    "https://www.artificialintelligence-news.com/2023/04/28/palantir-demos-how-ai-can-used-military/",
    "https://www.artificialintelligence-news.com/2023/04/26/ftc-chairwoman-no-ai-exemption-to-existing-laws/",
    "https://www.artificialintelligence-news.com/2023/04/24/bill-gates-ai-teaching-kids-literacy-within-18-months/",
    "https://www.artificialintelligence-news.com/2023/04/21/google-creates-new-ai-division-to-challenge-openai/"
]

session = requests.Session()
pages_content = []

for url in article_urls:
    try:
        time.sleep(2)
        response = session.get(url, headers=headers, timeout=10)

        if response.status_code == 200:
            article = Article(url)
            article.download()
            article.parse()
            pages_content.append({'url': url, 'text': article.text})
        
        else:
            print(f"Failed to fetch article at {url}")

    except Exception as e:
        print(f"Error occurred while fetching article at {url}: {e}")

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")

user = "veaceslavcalestru"
dataset = "langchain-autoagents"
dataset_path = f"hub://{user}/{dataset}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

Your Deep Lake dataset has been successfully created!
The dataset is private so make sure you are logged in!


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

all_texts = []
for d in pages_content:
    chunks = text_splitter.split_text(d['text'])
    for chunk in chunks:
        all_texts.append(chunk)

In [4]:
db.add_texts(all_texts)

/

Dataset(path='hub://veaceslavcalestru/langchain-autoagents', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (94, 1536)  float32   None   
    id        text      (94, 1)      str     None   
 metadata     json      (94, 1)      str     None   
   text       text      (94, 1)      str     None   


['87831017-7572-11ee-b60b-cc4740c98b6b',
 '87831018-7572-11ee-98a7-cc4740c98b6b',
 '87831019-7572-11ee-b082-cc4740c98b6b',
 '8783101a-7572-11ee-b8f9-cc4740c98b6b',
 '8783101b-7572-11ee-9aea-cc4740c98b6b',
 '8783101c-7572-11ee-8297-cc4740c98b6b',
 '8783101d-7572-11ee-9092-cc4740c98b6b',
 '8783101e-7572-11ee-bdca-cc4740c98b6b',
 '8783101f-7572-11ee-a0b8-cc4740c98b6b',
 '87831020-7572-11ee-9f19-cc4740c98b6b',
 '87831021-7572-11ee-99ea-cc4740c98b6b',
 '87831022-7572-11ee-bac4-cc4740c98b6b',
 '87831023-7572-11ee-b229-cc4740c98b6b',
 '87831024-7572-11ee-b51c-cc4740c98b6b',
 '87831025-7572-11ee-9369-cc4740c98b6b',
 '87831026-7572-11ee-bdf0-cc4740c98b6b',
 '87831027-7572-11ee-9e80-cc4740c98b6b',
 '87831028-7572-11ee-96ea-cc4740c98b6b',
 '87831029-7572-11ee-a556-cc4740c98b6b',
 '8783102a-7572-11ee-b910-cc4740c98b6b',
 '8783102b-7572-11ee-ba1e-cc4740c98b6b',
 '8783102c-7572-11ee-8dfa-cc4740c98b6b',
 '8783102d-7572-11ee-a04c-cc4740c98b6b',
 '8783102e-7572-11ee-88e9-cc4740c98b6b',
 '8783102f-7572-

In [5]:
retriever = db.as_retriever()
retriever.search_kwargs['k'] = 3

CUSTOM_TOOL_DOCS_SEPARATOR ="\n---------------\n"

def retrieve_n_docs_tool(query: str) -> str:
    """Searches for relevant documents that may contain the answer to the query."""
    docs = retriever.get_relevant_documents(query)
    texts = [doc.page_content for doc in docs]
    texts_merged = "---------------\n" + CUSTOM_TOOL_DOCS_SEPARATOR.join(texts) + "\n---------------"
    return texts_merged

In [6]:
from langchain.agents.tools import Tool

tools = [
    Tool(
        name="Search Private Docs",
        func=retrieve_n_docs_tool,
        description="useful for when you need to answer questions about current events about Artificial Intelligence"
    )
]

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner

model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [8]:
response = agent.run("Write an overview of Artificial Intelligence regulations by governments by country")



> Entering new  chain...
steps=[Step(value='Research and gather information on the current state of Artificial Intelligence (AI) regulations by governments in different countries.'), Step(value='Organize the information by country and categorize it based on the level of AI regulation.'), Step(value='Provide an overview of the AI regulations in each country, including any specific laws or policies that have been implemented.'), Step(value='Include information on the key areas covered by the regulations, such as data privacy, algorithmic transparency, liability, and ethical considerations.'), Step(value='Highlight any notable differences or similarities between the regulations in different countries.'), Step(value='Summarize the overall trends and developments in AI regulations globally.'), Step(value='Given the above steps taken, provide an overview of Artificial Intelligence regulations by governments by country. ')]

> Entering new  chain...
Action:
```
{
  "action": "Search Private

In [9]:
print(response)

Based on the previous steps and the information provided, it seems that the AI regulations by governments in different countries can be summarized as follows:

1. European Union (EU):
   - The European Parliament has recently endorsed new transparency and risk-management rules for AI systems known as the AI Act.
   - The rules aim to ensure the safety, transparency, traceability, and non-discrimination of AI systems.
   - Exemptions have been added for research activities and AI components provided under open-source licenses.
   - Regulatory sandboxes are being promoted to test AI before deployment.

2. United States (US):
   - The US follows a tech approach of experimenting first and then retrofitting to other markets and regulatory frameworks.
   - US-based AI developers may have an advantage in terms of innovation due to this approach.

3. United Kingdom (UK):
   - The UK is adopting a similar approach to the US, but due to its proximity to the EU market, UK-based developers are mor